Part 1 Section 2 - Normalize and Clean Data

In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv("data.csv")
# Get a warning that columns 0, 4, 5, 6 and 7 have mixed types. May need to clean this later
df.head()

C:\Users\jpo-0\AppData\Local\Temp\ipykernel_1652\3136517679.py:1: DtypeWarning: Columns (0,4,5,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data.csv")


,Id,Title,Company,Date,Location,Area,Classification,SubClassification,Requirement,FullDescription,LowestSalary,HighestSalary,JobType
0,37404348,Casual Stock Replenisher,Aldi Stores,2018-10-07T00:00:00.000Z,Sydney,North West & Hills District,Retail & Consumer Products,Retail Assistants,Our Casual Stock Replenishers pride themselves...,NaN,0,30,NaN
1,37404337,Casual Stock Replenisher,Aldi Stores,2018-10-07T00:00:00.000Z,Richmond & Hawkesbury,NaN,Retail & Consumer Products,Retail Assistants,Our Casual Stock Replenishers pride themselves...,NaN,0,30,NaN
2,37404356,RETAIL SALES SUPERSTARS and STYLISTS Wanted - ...,LB Creative Pty Ltd,2018-10-07T00:00:00.000Z,Brisbane,CBD & Inner Suburbs,Retail & Consumer Products,Retail Assistants,BRAND NEW FLAGSHIP STORE OPENING - SUNSHINE PLAZA,NaN,0,30,NaN
3,37404330,Team member - Belrose,Anaconda Group Pty Ltd,2018-10-07T00:00:00.000Z,Gosford & Central Coast,NaN,Retail & Consumer Products,Retail Assistants,Bring it on - do you love the great outdoors a...,NaN,0,30,NaN
4,37404308,"Business Banking Contact Centre Specialist, Ni...",Commonwealth Bank - Business & Private Banking,2018-10-07T00:00:00.000Z,Sydney,Ryde & Macquarie Park,Call Centre & Customer Service,Sales - Inbound,"We are seeking highly articulate, enthusiastic...",NaN,0,30,NaN


Step 1 - Create Average Salary column

In [4]:
df['AverageSalary'] = df[['LowestSalary', 'HighestSalary']].mean(axis=1)

Step 2 - Fix ID column inconsistences

In [6]:
# First a new column will be assigned to the cleaned Id column to ensure original data is not disturbed in cleaning process

# Now match all digits except the first 8 using lookbehind match Regex
df['NewId'] = df['Id'].replace(to_replace=r'(?<=\d{8}).*', regex=True, value='')

# This has now worked so set Id to new Id and remove new column
df['Id'] = df['NewId']
df = df.drop(['NewId'], axis = 1)


Id                                                            37905269
Title                Multiple SAP ABAP Developers - Federal Government
Company                                               HiTech Personnel
Date                                          2018-12-10T00:00:00.000Z
Location                                                     Melbourne
Area                                               CBD & Inner Suburbs
Classification                  Information & Communication Technology
SubClassification                               Developers/Programmers
Requirement          FEDERAL GOVT - MELBOURNE CBD - Multiple SAP AB...
FullDescription      <ul>\n            <li><strong>High profile FED...
LowestSalary                                                       150
HighestSalary                                                      200
JobType                                                  Contract/Temp
AverageSalary                                                    175.0
Name: 

Step 3 - Fix Date column

In [20]:
df['Date'] = df['Date'].replace(to_replace=r'(?<=^\d{4}-\d{2}-\d{2}).*', regex=True, value='')
df.head()

,Id,Title,Company,Date,Location,Area,Classification,SubClassification,Requirement,FullDescription,LowestSalary,HighestSalary,JobType,AverageSalary
0,37404348,Casual Stock Replenisher,Aldi Stores,2018-10-07,Sydney,North West & Hills District,Retail & Consumer Products,Retail Assistants,Our Casual Stock Replenishers pride themselves...,NaN,0,30,NaN,15.0
1,37404337,Casual Stock Replenisher,Aldi Stores,2018-10-07,Richmond & Hawkesbury,NaN,Retail & Consumer Products,Retail Assistants,Our Casual Stock Replenishers pride themselves...,NaN,0,30,NaN,15.0
2,37404356,RETAIL SALES SUPERSTARS and STYLISTS Wanted - ...,LB Creative Pty Ltd,2018-10-07,Brisbane,CBD & Inner Suburbs,Retail & Consumer Products,Retail Assistants,BRAND NEW FLAGSHIP STORE OPENING - SUNSHINE PLAZA,NaN,0,30,NaN,15.0
3,37404330,Team member - Belrose,Anaconda Group Pty Ltd,2018-10-07,Gosford & Central Coast,NaN,Retail & Consumer Products,Retail Assistants,Bring it on - do you love the great outdoors a...,NaN,0,30,NaN,15.0
4,37404308,"Business Banking Contact Centre Specialist, Ni...",Commonwealth Bank - Business & Private Banking,2018-10-07,Sydney,Ryde & Macquarie Park,Call Centre & Customer Service,Sales - Inbound,"We are seeking highly articulate, enthusiastic...",NaN,0,30,NaN,15.0


Step 4 - Change Date and Id Column Types

In [8]:
df["Date"] = pd.to_datetime(df["Date"])
df["Id"] = pd.to_numeric(df["Id"])
df.dtypes

Id                                 int64
Title                             object
Company                           object
Date                 datetime64[ns, UTC]
Location                          object
Area                              object
Classification                    object
SubClassification                 object
Requirement                       object
FullDescription                   object
LowestSalary                       int64
HighestSalary                      int64
JobType                           object
AverageSalary                    float64
dtype: object

Step 5 - Find and Fix Duplicate Data

In [9]:
df[df.duplicated()].shape[0]

108

There are 108 duplicate rows, can fix this by using the inbuilt pandas function drop_duplicates

In [10]:
df = df.drop_duplicates()
df[df.duplicated()].shape[0]

0

Now there are no more whole row duplicates - we will check for duplicates in the only row that should be unique (Id)

In [12]:
df.duplicated(subset=['Id']).sum()
# By going through it appears after fixing the Ids it creates 108 duplicate whole rows
# We can then drop the whole duplicate rows but we find there are still 13898 duplicate Ids

# Alternatively after fixing rows then changing Id to numeric there are 14165 duplicate Ids
# Then when we drop duplicate whole rows this goes down to 14057


14057

Can see there are still duplicates in the data set. We will find them and see whats going on

In [14]:
df.loc[df.duplicated(subset='Id')]
# Found many rows with duplicate Ids. We will take one and find its duplicate. Will use Id 37709736
df.loc[df['Id'] == 37709736]

,Id,Title,Company,Date,Location,Area,Classification,SubClassification,Requirement,FullDescription,LowestSalary,HighestSalary,JobType,AverageSalary
149618,37709736,Senior Account Manager,NaN,2018-11-13 00:00:00+00:00,Gold Coast,NaN,"Advertising, Arts & Media",Agency Account Management,We're searching for a talented and energetic A...,<p>As one of the Gold Coast's largest agencies...,0,30,Full Time,15.0
154461,37709736,Senior Account Manager,NaN,2018-12-11 00:00:00+00:00,Gold Coast,NaN,"Advertising, Arts & Media",Agency Account Management,We're searching for a talented and energetic A...,NaN,0,30,NaN,15.0


Can see there are still duplicate listings. The difference being one row has NaN FullDescription and JobType whereas the other row contains this information


My current idea now is.. If df['Id'] is duplicate, test if it has NaN description or Jobtype (may be other categories too do some testing on the second testing thing). Drop the NaN versions